<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/06b_GPT2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Import libraries, make sure you don't reinstall gpt_2_simple if you have already sone so by commenting that line

In [2]:
%tensorflow_version 1.x
#!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Load and check the data
you can download "all_speech_sentence_filtered.pkl" from [here](https://drive.google.com/file/d/1AwSUBntWHP6udrHMpl9CF9hEHmarNyhF/view?usp=sharing).

In [11]:
speeches = pd.read_pickle("all_speech_sentence_filtered.pkl")

print("Number of speeches: {}".format(len(speeches)))
speeches = speeches.head(100000)

Number of speeches: 298792


Make .txt files for republican and democrat data as gpt_2_simple needs the data in this format

In [12]:
# make separate txt files for republicans and democrats
with open("rep_speeches.txt", 'w') as f1, open("dem_speeches.txt", 'w') as f2:
  for i in range(len(speeches)):
    speech = speeches.iloc[i].Questions
    party = speeches.iloc[i].Party
    if party == 'R':
      f1.write(speech + '\n')
    else:
      f2.write(speech + '\n')

# check files 
with open('rep_speeches.txt', 'r') as f:
  lines = f.readlines()
  print(len(lines))
  for line in lines[:5]:
    print(line)

46666
as all Members are aware, I have been working for some time on various rules changes that would ensure a more efficient process by which the Senate considers appropriations bills. One of our concerns has been reinstating rule XVI with respect to legislation on appropriations bills, I believe that many of the extraneous items that have been added to appropriations bills over the past few Congresses would have been ruled out of order if the Senate still had the ability to raise a point of order against legislation on appropriations bills formerly contained in rule XVI. Other rule abuses occurred during the closing days of the 105th Congress. Consequently, I will shortly introduce five Senate resolutions regarding rules and budget process changes and will ask for their proper referral. Once the resolutions have been referred to the appropriate committee, it is my hope the chairmen of the committees will begin swift committee work on the resolutions so the committees can act on the c

Finetunine GPT-2

In [7]:
# download the GPT-2 model if you have not done so
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 224Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 73.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 230Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 169Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 320Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 139Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 129Mit/s]                                                       


In [3]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Train for republican data

In [7]:
file_name = "rep_speeches.txt"

In [8]:
RUN_NAME = 'rep'
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=2000,
              restore_from='fresh',
              run_name=RUN_NAME,
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [03:12<00:00, 192.47s/it]


dataset has 31956684 tokens
Training...
[10 | 52.16] loss=3.33 avg=3.33
[20 | 97.84] loss=3.08 avg=3.20
[30 | 143.52] loss=3.14 avg=3.18
[40 | 189.24] loss=3.21 avg=3.19
[50 | 234.84] loss=3.10 avg=3.17
[60 | 280.49] loss=3.06 avg=3.15
[70 | 326.07] loss=3.19 avg=3.16
[80 | 371.69] loss=3.12 avg=3.15
[90 | 417.36] loss=2.96 avg=3.13
[100 | 463.02] loss=3.17 avg=3.13
[110 | 508.64] loss=3.15 avg=3.14
[120 | 554.25] loss=3.01 avg=3.13
[130 | 599.93] loss=3.02 avg=3.12
[140 | 645.54] loss=3.11 avg=3.12
[150 | 691.20] loss=2.95 avg=3.10
[160 | 736.83] loss=2.99 avg=3.10
[170 | 782.49] loss=2.96 avg=3.09
[180 | 828.20] loss=3.01 avg=3.08
[190 | 873.92] loss=2.97 avg=3.08
[200 | 919.63] loss=3.17 avg=3.08
======== SAMPLE 1 ========
. There are few people who understand the difference between a college education and a job at a company that is not in your field, and there are many employees who have earned their education at a company that provides employment for many other employees. Therefor

Make sure you restart runtime before running this cell!

In [3]:
RUN_NAME = 'rep'
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=RUN_NAME)

Loading checkpoint checkpoint/rep/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/rep/model-2000


Generate republican answers
You can download the weights from [here](https://drive.google.com/file/d/1qRdGcykWvwE0WHPC1LGrDbE-1DwSMPuc/view).

In [4]:
questions = ["Should abortion be illegal?",
             "What do you believe about tax increases?",
             "Should same-sex marriage be legal?",
             "Is climate change real?",
             "Should immigrants be allowed to obtain citizenship?",
             "Who should be given voting rights?",
             "Should we have higher taxes for higher incomes?",
             "Should we allow death penalty?",
             "Should we have universal healthcare?",
             "Do government regulations hinder free market capitalism?",
             "What do you think about the current president?",
             "Should we reduce national debt?",
             "Should we increase spending on healthcare?",
             "Should we increase spending on education?",
             "Should every American have equal opportunities regardless of sex, age and race?",
             "Should be introduce more gun control measures?",
             "What do you think about the current president?",
             "Should Americans be free?",
             "What party do you support?",
             "What is the biggest threat to America?"
             ]

In [5]:
with open("rep_generated.txt", 'w') as f:
  for i in range(len(questions)):
    text = gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix=questions[i],
              nsamples=1,
              batch_size=1,
              return_as_list=True)[0]
    print(text)
    f.write(text + '\n')

Should abortion be illegal? We agree with the Supreme Court that abortion should be legal. Congress should not be at war with abortion. In fact, I believe that there should be a preemption of the law. But the question becomes, what is necessary for the Supreme Court to decide? The Supreme Court has already ruled in Roe v. Wade that abortion is not an act of violence, that it is a legitimate social and medical procedure, and that the law is not in conflict with the Constitution. We need to do something about it. We have a chance to do that. But we have to do it in a way that does not give the American people the opportunity to stop it. That, is what the Constitution requires. It is not a decision by the Federal Government. It is not a decision by the States. It is a decision by the people of the States. We need to do this. We need to do it with a sound decision. So, yes, we are going to have to do it. But we are not going to be able to do it. We need to do it in a way that is consistent

Train for democrat data
You can download the weights from [here](https://drive.google.com/file/d/1qRdGcykWvwE0WHPC1LGrDbE-1DwSMPuc/view).

In [5]:
file_name = "dem_speeches.txt"

sess = gpt2.start_tf_sess()
RUN_NAME = 'dem'

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=2000,
              restore_from='fresh',
              run_name=RUN_NAME,
              print_every=10,
              sample_every=500,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [04:06<00:00, 246.93s/it]


dataset has 36883131 tokens
Training...
[10 | 53.17] loss=3.19 avg=3.19
[20 | 99.40] loss=3.07 avg=3.13
[30 | 145.83] loss=2.92 avg=3.06
[40 | 192.03] loss=3.13 avg=3.08
[50 | 238.33] loss=3.10 avg=3.08
[60 | 284.63] loss=3.00 avg=3.07
[70 | 330.99] loss=3.03 avg=3.06
[80 | 377.20] loss=3.13 avg=3.07
[90 | 423.42] loss=3.07 avg=3.07
[100 | 469.55] loss=3.08 avg=3.07
[110 | 515.76] loss=3.10 avg=3.08
[120 | 561.83] loss=3.03 avg=3.07
[130 | 607.95] loss=3.05 avg=3.07
[140 | 654.15] loss=3.01 avg=3.06
[150 | 700.33] loss=3.01 avg=3.06
[160 | 746.31] loss=3.10 avg=3.06
[170 | 792.40] loss=3.02 avg=3.06
[180 | 838.38] loss=3.06 avg=3.06
[190 | 884.46] loss=2.96 avg=3.06
[200 | 930.49] loss=3.27 avg=3.07
[210 | 976.53] loss=3.04 avg=3.07
[220 | 1022.70] loss=3.16 avg=3.07
[230 | 1068.85] loss=3.03 avg=3.07
[240 | 1114.93] loss=3.14 avg=3.07
[250 | 1161.04] loss=2.90 avg=3.06
[260 | 1207.23] loss=3.06 avg=3.06
[270 | 1253.40] loss=3.09 avg=3.07
[280 | 1299.42] loss=3.13 avg=3.07
[290 | 1345.

Make sure you restart runtime before running this cell!

In [3]:
RUN_NAME = 'dem'
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=RUN_NAME)

Loading checkpoint checkpoint/dem/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/dem/model-2000


Generate democrat answers

In [4]:
questions = ["Should abortion be illegal?",
             "What do you believe about tax increases?",
             "Should same-sex marriage be legal?",
             "Is climate change real?",
             "Should immigrants be allowed to obtain citizenship?",
             "Who should be given voting rights?",
             "Should we have higher taxes for higher incomes?",
             "Should we allow death penalty?",
             "Should we have universal healthcare?",
             "Do government regulations hinder free market capitalism?",
             "What do you think about the current president?",
             "Should we reduce national debt?",
             "Should we increase spending on healthcare?",
             "Should we increase spending on education?",
             "Should every American have equal opportunities regardless of sex, age and race?",
             "Should be introduce more gun control measures?",
             "What do you think about the current president?",
             "Should Americans be free?",
             "What party do you support?",
             "What is the biggest threat to America?"
             ]

In [6]:
with open("dem_generated.txt", 'w') as f:
  for i in range(len(questions)):
    text = gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix=questions[i],
              nsamples=1,
              batch_size=1,
              return_as_list=True)[0]
    print(text)
    f.write(text + '\n')

Should abortion be illegal? Should we not require a woman to have an abortion if she has been convicted of a crime and the State determines that the judgment of the State is too hard to achieve? I am not a Republican. Democrats, or Republicans here in this House. We are not a party of the death penalty. We are not a party of the death penalty. We are not a party of the transience of circumstances. We are not a party of the death penalty. But we are a party of the death penalty. We are a party of the death penalty. We are a party of the death penalty. It is a crime under the law of the United States to kill a human being, I do not believe that the United States should, under the current law of the United States, require that the people of the United States be deprived of the right to an abortion. The United States should not be deprived of the right to an abortion under the Constitution. It is a crime under the Constitution to kill a human being. Under the current law of the United Stat